# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [23]:
# Instalar o selenium
#!pip install selenium
# baixar o webdriver 
#(já está na mesma pasta do código "Aula 3")


#importar do selenium o webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
#criar o navegador
navegador = webdriver.Chrome("chromedriver.exe")


# Passo 1: Pegar cotação do Dolar

# - entrar no site do google
navegador.get("https://google.com")
# - pesquisar "cotação do dolar"
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# - pegar a cotação da página do google

cotacao_dolar = navegador.find_element(By.XPATH,
                       '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

# Passo 2: Pegar cotação do Euro
# - entrar no site do google
navegador.get("https://google.com")
# - pesquisar "cotação do euro"
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# - pegar a cotação da página do google

cotacao_euro = navegador.find_element(By.XPATH,
                       '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)
# Passo 3: Pegar cotação do Ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
# - pegar cotação do ouro do melhor cambio
cotacao_ouro = navegador.find_element(By.XPATH,
                       '//*[@id="comercial"]').get_attribute('value')
#padronizar cotação do ouro com . inves de ,
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)





C:\Users\User\AppData\Local\Temp\ipykernel_21372\3482003846.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navegador = webdriver.Chrome("chromedriver.exe")


5.3797
5.1631670750000005
282.87


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [24]:
# Passo 4: Importar a base de dados
#sempre que for importar uma base de dados para o python, importar o pandas as pd

import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [26]:
# Filtrar a tabela de acordo com a coluna "Moeda" na planilha acima

# Passo 5: Atualizar as cotação
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# Passo 5: Atualizar preço de compra = Preço Original * Cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]


# Passo 5: Atualizar preço de venda = Preço de Compra * Margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

# tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R${:.2f}".format)

display(tabela)


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.379700,5379.646203,1.40,R$7531.50
1,Carro Renault,4500.00,Euro,5.163167,23234.251838,2.00,R$46468.50
2,Notebook Dell,899.99,Dólar,5.379700,4841.676203,1.70,R$8230.85
3,IPhone,799.00,Dólar,5.379700,4298.380300,1.70,R$7307.25
4,Carro Fiat,3000.00,Euro,5.163167,15489.501225,1.90,R$29430.05
5,Celular Xiaomi,480.48,Dólar,5.379700,2584.838256,2.00,R$5169.68
6,Joia 20g,20.00,Ouro,282.870000,5657.400000,1.15,R$6506.01


### Agora vamos exportar a nova base de preços atualizada

In [1]:
# Passo 6: Exportar o relatório atualizado
#index=False serve pra tirar a primeira coluna que numera as linhas
tabela.to_excel("Produtos.xlsx", index=False)
navegador.quit()


NameError: name 'tabela' is not defined

In [7]:
#importar do selenium o webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
#criar o navegador
navegador = webdriver.Chrome("chromedriver.exe")

import pyautogui
import pyperclip
import time


# - entrar no site da asc
navegador.get("https://isd1.algar.tech/flow/view/cod_flow/1302")


# - entrar na conta da asc
navegador.find_element(By.XPATH,
                       '//*[@id="login-user"]').send_keys("matheusncm")

navegador.find_element(By.XPATH,
                       '//*[@id="login-password"]').send_keys("Algar@2022")

navegador.find_element(By.XPATH,
                       '//*[@id="form-login"]/div[3]/div/button').send_keys(Keys.ENTER)


navegador.get("https://docs.google.com/spreadsheets/d/1tsD2nxeM6cKjFXrmUAHsWLOLXQzyJCMUSpt99wptG34/edit#gid=935239526")

C:\Users\User\AppData\Local\Temp\ipykernel_4592\2463805569.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navegador = webdriver.Chrome("chromedriver.exe")
